In [1]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=0

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=0


In [2]:
from transformers import AutoTokenizer
import sys
import json
from scipy.special import softmax
from transformers import AutoModelForSequenceClassification
from simpletransformers.classification import ClassificationModel, ClassificationArgs
from transformers import AutoModelForSequenceClassification, TextClassificationPipeline, AutoTokenizer, AutoConfig
import torch
import argparse
import numpy as np
import pandas as pd


/home/tajak/miniconda3/envs/emma_main/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Load the test datasets

test_en = pd.read_json("../../datasets/ParlaMint-EN-CAP-test-dataset.jsonl", lines=True)
test_hr = pd.read_json("../../datasets/ParlaMint-HR-CAP-test-dataset.jsonl", lines=True)
test_ba = pd.read_json("../../datasets/ParlaMint-BA-CAP-test-dataset.jsonl", lines=True)
test_sr = pd.read_json("../../datasets/ParlaMint-RS-CAP-test-dataset.jsonl", lines=True)

print(test_en.shape, test_hr.shape, test_ba.shape,test_sr.shape)

(876, 6) (869, 6) (824, 6) (874, 6)


In [5]:
def apply_classifier(test_df, df_test_name):
	from transformers import pipeline
	import time
	print("Loading the model ...")

	cap_labels = ["Education", "Technology", "Health", "Environment", "Housing", "Labor", "Defense", "Government Operations", "Social Welfare", "Other", "Macroeconomics", "Domestic Commerce", "Civil Rights", "International Affairs", "Transportation", "Immigration", "Law and Crime", "Agriculture", "Foreign Trade", "Culture", "Public Lands", "Energy"]

	cap_model = pipeline("text-classification", model="classla/ParlaCAP-Topic-Classifier", device=0, max_length=512, truncation=True)

	texts = test_df["text"].to_list()
	preds = []

	start_time = time.time()
	# Classify the texts
	results = cap_model(texts)
	end_time = time.time()
	elapsed_time_min = end_time-start_time

	print(f"Prediction finished. It took {elapsed_time_min/60} min for {test_df.shape[0]} instances - {elapsed_time_min/test_df.shape[0]} s per instance.")

	# Output the results
	for result in results:
		preds.append(result["label"])

	# Create a json with results

	current_results = {
		"system": "ParlaCAP-classifier-2nd-iteration",
		"predictions": [
			{
			"train": "ParlaCAP-train",
			"test": "{}".format(df_test_name),
			"predictions": preds,
			}
		]
		}

	# Save the results as a new json
	with open("submissions/submission-{}-{}.json".format("ParlaCAP-2", df_test_name), "w") as file:
		json.dump(current_results, file)

	print("Classification with {} on {} finished.".format("ParlaCAP-2", df_test_name))


In [6]:
apply_classifier(test_en, "ParlaCAP-EN-test")

Loading the model ...
Prediction finished. It took 0.3762852668762207 min for 876 instances - 0.025772963484672652 s per instance.
Classification with ParlaCAP-2 on ParlaCAP-EN-test finished.


In [7]:
apply_classifier(test_hr, "ParlaCAP-HR-test")

Loading the model ...
Prediction finished. It took 0.34687753121058146 min for 869 instances - 0.023950117229729444 s per instance.
Classification with ParlaCAP-2 on ParlaCAP-HR-test finished.


In [8]:
apply_classifier(test_sr, "ParlaCAP-RS-test")

Loading the model ...
Prediction finished. It took 0.36757518847783405 min for 874 instances - 0.025233994632345588 s per instance.
Classification with ParlaCAP-2 on ParlaCAP-RS-test finished.


In [9]:
apply_classifier(test_ba, "ParlaCAP-BA-test")

Loading the model ...
Prediction finished. It took 0.3366943717002869 min for 824 instances - 0.02451658046361312 s per instance.
Classification with ParlaCAP-2 on ParlaCAP-BA-test finished.
